## Imports

In [1]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()) if torch.cuda.is_available() else "No GPU")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

CUDA available: True
Device count: 1
Current device: 0
Device name: NVIDIA GeForce RTX 4090


In [ ]:
from transformers import AutoTokenizer
import sys
import os
import pandas as pd

sys.path.append(os.path.abspath(".."))

from tfm.ai_generated_detection.ai_generated_detection_models import (
    Wangkevin02AIDetectorModel,
    DesklibAIDetectorModel,
    PrasoonmhwrAIDetectorModel,
    FakespotAIDetectorModel,
    calculate_ai_human_metrics,
)

c:\Users\Alonso\Documents\## UPM\TFM\git\TFM\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Leer Datos

In [3]:
data = pd.read_csv("../data/diagnoses_df/ground_truth_df.csv")
data['generated'] = False
data

,Descripcion_diagnosticos,Descripcion_diagnosticos_limpio,Codigos_diagnosticos,Diagnosticos_estandar,generated
0,F30.2 - MANÍA CON SÍNTOMAS PSICÓTICOS.,f30.2 mania con sintomas psicoticos.,['F30.9'],"['Episodio maníaco, no especificado']",False
1,\r\nDescompensación rasgos patológicos de pers...,descompensacion rasgos patologicos de personal...,"['F60.3', 'F79.0']",['Trastorno de inestabilidad emocional de la p...,False
2,Intento de suicidio,intento de suicidio,"['F31.3', 'F32.9', 'T14.91']","['Trastorno bipolar, episodio actual depresivo...",False
3,ID: trastorno adaptativo,id trastorno adaptativo,['F43.2'],['Trastorno de adaptación'],False
4,JC: Episodio depresivo con intento autolitico ...,jc episodio depresivo con intento autolitico f...,"['F32.9', 'T14.91']","['Episodio depresivo sin especificación', 'Sui...",False
...,...,...,...,...,...
79043,"-Negativa a la ingesta, agitación y alteracion...",negativa a la ingesta agitacion y alteraciones...,"['F4.0', 'F41.1', 'F60.4', 'COGNITIV', 'F13.0']","['Trastorno de ansiedad, disociativo, relacion...",False
79044,"Eje I: \r\n-Psicosis no especificada, leve des...",eje i psicosis no especificada leve descompens...,"['F32.9', 'F33.9', 'F60.3', 'Z63', 'F12.0', 'F...","['Episodio depresivo sin especificación', 'Tra...",False
79045,T ansiedad con insomnio plurifactorial.,t ansiedad con insomnio plurifactorial.,"['F4.0', 'F51.0']","['Trastorno de ansiedad, disociativo, relacion...",False
79046,"- T. de ansiedad con insomnio comórbido, en re...",t. de ansiedad con insomnio comorbido en remis...,"['F4.0', 'F51.0']","['Trastorno de ansiedad, disociativo, relacion...",False


## Pruebas de modelos

La idea es probar el rendimiento de diferentes modelos de clasificación de textos en base a si son escritos por humanos o generados con inteligencia artificial.

Un modelo será usado para clasificar los textos generados, de manera que intertesará que clasifique los textos generados erróneamente, lo cual será un indicativo de que son de buena claidad (o de queson similares a los humanos). 

Para elegir un modelo para esta tarea, se probará el rendimiento de diferentes modelos evaluándolos sobre el conjunto de datos original, en el que todos los diagnósticos son humanos. Aquel modelo que consiga un mejor accuracy en este conjunto, será el seleccionado para la tarea de evaluación posterior, pues será el modelo que a priori será mejor capaz de detectar los textos humanos.

### wangkevin02/AI_Detect_Model

In [ ]:
classifier = Wangkevin02AIDetectorModel(model_name="wangkevin02/AI_Detect_Model")

In [ ]:
result = classifier.get_probability(data['Descripcion_diagnosticos'].to_list(), batch_size=64)
predicted_indices = torch.argmax(result, dim=1)
predicted_indices

Processing batches: 100%|██████████| 1236/1236 [46:02<00:00,  2.23s/it] 


In [ ]:
calculate_ai_human_metrics(predicted_indices, data)

Accuracy: 0.7073
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.71      0.83     79048
           1       0.00      0.00      0.00         0

    accuracy                           0.71     79048
   macro avg       0.50      0.35      0.41     79048
weighted avg       1.00      0.71      0.83     79048



c:\Users\Alonso\Documents\## UPM\TFM\git\TFM\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Alonso\Documents\## UPM\TFM\git\TFM\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Alonso\Documents\## UPM\TFM\git\TFM\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", re

### desklib/ai-text-detector-v1.01

In [ ]:
model_dir = "desklib/ai-text-detector-v1.01"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = DesklibAIDetectorModel.from_pretrained(model_dir)

In [ ]:
probs, labels = model.predict(
    data['Descripcion_diagnosticos'].to_list(),
    tokenizer=tokenizer,
    device=device,
    batch_size=16,
    max_len=768,
    threshold=0.5
)

Batches: 100%|██████████| 4941/4941 [20:30<00:00,  4.01it/s]  


In [ ]:
calculate_ai_human_metrics(torch.tensor(labels), data)

Accuracy: 0.2992
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.30      0.46     79048
           1       0.00      0.00      0.00         0

    accuracy                           0.30     79048
   macro avg       0.50      0.15      0.23     79048
weighted avg       1.00      0.30      0.46     79048



c:\Users\Alonso\Documents\## UPM\TFM\git\TFM\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Alonso\Documents\## UPM\TFM\git\TFM\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Alonso\Documents\## UPM\TFM\git\TFM\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", re

### prasoonmhwr/ai_detection_model

In [4]:
detector = PrasoonmhwrAIDetectorModel("prasoonmhwr/ai_detection_model")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at prasoonmhwr/ai_detection_model and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'roberta.embeddings.LayerNorm.bias', 'roberta.embeddings.LayerNorm.weight', 'roberta.embeddings.position_embeddings.weight', 'roberta.embeddings.token_type_embeddings.weight', 'roberta.embeddings.word_embeddings.weight', 'roberta.encoder.layer.0.attention.output.LayerNorm.bias', 'roberta.encoder.layer.0.attention.output.LayerNorm.weight', 'roberta.encoder.layer.0.attention.output.dense.bias', 'roberta.encoder.layer.0.attention.output.dense.weight', 'roberta.encoder.layer.0.attention.self.key.bias', 'roberta.encoder.layer.0.attention.self.key.weight', 'roberta.encoder.layer.0.attention.self.query.bias', 'roberta.encoder.layer.0.attention.self.query.weight', 'roberta.encoder.layer.0.attention.self.value.bias', 'roberta.encod

In [5]:
probs, labels = detector.predict(data['Descripcion_diagnosticos'].to_list(), batch_size=16)

Predicting: 100%|██████████| 4941/4941 [01:14<00:00, 65.98it/s]


In [6]:
calculate_ai_human_metrics(torch.tensor(labels), data)

Accuracy: 0.9888


### fakespot-ai/roberta-base-ai-text-detection-v1

In [4]:
classifier = FakespotAIDetectorModel()

Device set to use cuda:0


In [ ]:
preds = classifier.predict(data)

Clasificando: 100%|██████████| 2471/2471 [06:47<00:00,  6.07it/s]


In [7]:
calculate_ai_human_metrics(torch.tensor(preds), data)

Accuracy: 0.7307
